<a href="https://colab.research.google.com/github/K-Himanshu/DataAnalysis/blob/main/covidDataAnalysisusingSql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [50]:
import pandas as pd

In [51]:
import sqlite3
from sqlalchemy import create_engine

In [52]:

engine=create_engine('sqlite://', echo=False)
file=pd.read_excel("/content/drive/MyDrive/owid-covid-data.xlsx")



In [53]:
file.head()


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [54]:
file.shape

(155139, 67)

In [55]:
file.drop(file.iloc[:, 17:67], inplace = True, axis = 1)


In [56]:
file.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN


In [57]:
file.drop(columns=['new_cases_smoothed', 'new_deaths_smoothed', 'new_cases_smoothed_per_million', 'new_deaths_smoothed_per_million','reproduction_rate'])


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,0.126,0.126,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,0.126,0.000,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,0.126,0.000,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,0.126,0.000,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,0.126,0.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
155134,ZWE,Africa,Zimbabwe,2022-01-12,224433.0,1433.0,5215.0,35.0,14870.823,94.950,345.543,2.319
155135,ZWE,Africa,Zimbabwe,2022-01-13,225084.0,651.0,5222.0,7.0,14913.958,43.135,346.007,0.464
155136,ZWE,Africa,Zimbabwe,2022-01-14,225637.0,553.0,5238.0,16.0,14950.599,36.642,347.067,1.060
155137,ZWE,Africa,Zimbabwe,2022-01-15,225637.0,0.0,5238.0,0.0,14950.599,0.000,347.067,0.000


In [58]:
file=file.dropna(subset=['iso_code','continent','location','date','total_cases','new_cases','total_deaths', 'new_deaths','total_cases_per_million','new_cases_per_million','total_deaths_per_million'])

In [59]:
file

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate
28,AFG,Asia,Afghanistan,2020-03-23,40.0,6.0,2.143,1.0,1.0,0.143,1.004,0.151,0.054,0.025,0.025,0.004,NaN
29,AFG,Asia,Afghanistan,2020-03-24,42.0,2.0,2.286,1.0,0.0,0.143,1.054,0.050,0.057,0.025,0.000,0.004,NaN
30,AFG,Asia,Afghanistan,2020-03-25,74.0,32.0,6.857,1.0,0.0,0.143,1.858,0.803,0.172,0.025,0.000,0.004,NaN
31,AFG,Asia,Afghanistan,2020-03-26,80.0,6.0,7.714,2.0,1.0,0.286,2.008,0.151,0.194,0.050,0.025,0.007,NaN
32,AFG,Asia,Afghanistan,2020-03-27,91.0,11.0,9.571,2.0,0.0,0.286,2.284,0.276,0.240,0.050,0.000,0.007,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155134,ZWE,Africa,Zimbabwe,2022-01-12,224433.0,1433.0,768.000,5215.0,35.0,17.571,14870.823,94.950,50.887,345.543,2.319,1.164,0.63
155135,ZWE,Africa,Zimbabwe,2022-01-13,225084.0,651.0,700.857,5222.0,7.0,16.286,14913.958,43.135,46.438,346.007,0.464,1.079,0.62
155136,ZWE,Africa,Zimbabwe,2022-01-14,225637.0,553.0,622.143,5238.0,16.0,14.571,14950.599,36.642,41.223,347.067,1.060,0.965,0.63
155137,ZWE,Africa,Zimbabwe,2022-01-15,225637.0,0.0,531.286,5238.0,0.0,12.857,14950.599,0.000,35.203,347.067,0.000,0.852,NaN


In [60]:
file.to_sql('covid', engine, if_exists='replace', index=False)
results=engine.execute('select * from covid order by total_cases')

results=pd.DataFrame(results, columns=file.columns)


In [61]:
results.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate
0,CYM,North America,Cayman Islands,2020-03-16,1.0,0.0,NaN,1.0,1.0,NaN,15.038,0.000,NaN,15.038,15.038,NaN,NaN
1,CYM,North America,Cayman Islands,2020-03-17,1.0,0.0,NaN,1.0,0.0,NaN,15.038,0.000,NaN,15.038,0.000,NaN,NaN
2,CYM,North America,Cayman Islands,2020-03-18,1.0,0.0,0.143,1.0,0.0,0.143,15.038,0.000,2.148,15.038,0.000,2.148,NaN
3,GUY,South America,Guyana,2020-03-12,1.0,1.0,NaN,1.0,1.0,NaN,1.265,1.265,NaN,1.265,1.265,NaN,NaN
4,GUY,South America,Guyana,2020-03-13,1.0,0.0,NaN,1.0,0.0,NaN,1.265,0.000,NaN,1.265,0.000,NaN,NaN


In [62]:
res=engine.execute('select total_cases, total_deaths, (total_deaths/total_cases)*100 as death_percentage  from covid where location <> "World" group by total_cases')
res=pd.DataFrame(res, columns=['total_cases', 'total_deaths','death_percentage'])

In [63]:
res.head()

,total_cases,total_deaths,death_percentage
0,1.0,1.0,100.000000
1,2.0,1.0,50.000000
2,3.0,1.0,33.333333
3,4.0,1.0,25.000000
4,5.0,1.0,20.000000


In [64]:
res=engine.execute('select location, total_cases, total_deaths, (total_deaths/total_cases)*100 as death_percentage  from covid where location like "%India" group by total_cases order by 3 desc')
res=pd.DataFrame(res, columns=['location', 'total_cases', 'total_deaths','death_percentage'])


In [65]:
res.head()

,location,total_cases,total_deaths,death_percentage
0,India,37380253.0,486451.0,1.301358
1,India,37122164.0,486066.0,1.309369
2,India,36850962.0,485752.0,1.318153
3,India,36317927.0,485131.0,1.335789
4,India,35875790.0,484213.0,1.349693


In [80]:
res=engine.execute('select location, total_cases as HighestInfectionCount from covid group by total_cases order by total_cases desc')
res=pd.DataFrame(res, columns=['location', 'HighestInfectionCount'])


In [81]:
res

,location,HighestInfectionCount
0,United States,65699947.0
1,United States,65445397.0
2,United States,64920604.0
3,United States,64062060.0
4,United States,63285605.0
...,...,...
74974,Zimbabwe,5.0
74975,Vanuatu,4.0
74976,Zimbabwe,3.0
74977,Sudan,2.0


In [86]:
res=engine.execute('select location, total_deaths as totalDeathCount from covid group by 1 order by 2 desc')
res=pd.DataFrame(res, columns=['location', 'totalDeathCount'])

In [87]:
res

,location,totalDeathCount
0,United States,850605.0
1,Brazil,621327.0
2,India,486451.0
3,Russia,314838.0
4,Mexico,301410.0
...,...,...
197,Anguilla,6.0
198,Bhutan,3.0
199,Greenland,3.0
200,Montserrat,1.0


In [ ]:
res=engine.execute('select continent, total_deaths from covid group by 1 order by 2 desc limit 5')
res=pd.DataFrame(res, columns=['continent', 'total_deaths'])

In [94]:
res

,continent,total_deaths
0,North America,850605.0
1,Europe,152109.0
2,South America,5380.0
3,Africa,5247.0
4,Asia,1990.0
